In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [2]:
# !pip install -U langchain-teddynote

## 인스턴스 생성

In [2]:
from swarm import Swarm, Agent

client = Swarm()

## Agent 기본 설정

- `name`: 에이전트 이름
- `instructions`: 에이전트 의 역할

In [ ]:
# 에이전트 생성
agent = Agent(
    name="Agent",
    instructions="You are a helpful agent. Your name is `테디`. You only speak Korean. Start your response with `안녕하세요!` + your name.",
)

# 메시지 생성
messages = [{"role": "user", "content": "Hi!"}]
response = client.run(agent=agent, messages=messages)

print(response.messages[-1]["content"])

## Handoff(전달 기능)

한 에이전트에서 다른 에이전트로 대화를 넘기는 방법을 보여줍니다.

In [5]:
from swarm import Swarm, Agent

# 인스턴스 생성
client = Swarm()

# 에이전트 생성
english_agent = Agent(
    name="English Agent",
    instructions="You only speak English.",
)

# 에이전트 생성
korean_agent = Agent(
    name="Korean Agent",
    instructions="You only speak Korean.",
)


# 함수 생성(function)
def transfer_to_korean_agent():
    """Transfer Korean speaking users immediately."""
    return korean_agent


# functions 에 함수 추가
english_agent.functions.append(transfer_to_korean_agent)

`english_agent` 에 메시지 추가: 한국어 입력시 `transfer_to_spanish_agent` 호출, korean_agent 가 반환

즉, `transfer_to_spanish_agent` 함수가 일종의 `Router` 역할을 합니다.

In [ ]:
# 메시지 전달
messages = [{"role": "user", "content": "안녕하세요?"}]

# 응답 받기
response = client.run(agent=english_agent, messages=messages)

print(response.messages[-1]["content"])

In [ ]:
from langchain_teddynote.messages import display_message_tree

display_message_tree(response.messages)

## Context Variables

In [ ]:
from swarm import Swarm, Agent

# 인스턴스 생성
client = Swarm()


def instructions(context_variables):
    xml_string = ""
    for key, value in context_variables.items():
        xml_string += f"<{key}>{value}</{key}>"
    return f"""You are a helpful agent.
Introduce yourself with referring to following personal information.
(You may skip if you don't have any information.)

#Here is your personal information: \n{xml_string}"""


# 에이전트 생성
agent = Agent(
    name="Agent",
    instructions=instructions,
)

# 컨텍스트 변수 생성
context_variables = {"name": "테디", "job": "AI Researcher", "company": "테디노트"}

# 응답 받기
response = client.run(
    messages=[{"role": "user", "content": "안녕하세요?"}],
    agent=agent,
    context_variables=context_variables,
)

print(response.messages[-1]["content"])

## Function Calling

In [ ]:
from swarm import Swarm, Agent

client = Swarm()


def get_weather(location: str) -> str:
    """Get the weather of a given location."""
    return "{'temp':20, 'unit':'C'}"


agent = Agent(
    name="Agent",
    instructions="You are a helpful agent.",
    functions=[get_weather],
)

messages = [{"role": "user", "content": "서울의 날씨는 어때?"}]

response = client.run(agent=agent, messages=messages)
print(response.messages[-1]["content"])

In [ ]:
from langchain_teddynote.messages import display_message_tree

display_message_tree(response.messages)

## 대화 루프

In [ ]:
from swarm import Swarm, Agent

client = Swarm()

my_agent = Agent(
    name="Agent",
    instructions="You are a helpful agent.",
)


def pretty_print_messages(messages):
    if len(messages) > 0:
        print(f"{messages[-1]['role']}: {messages[-1]['content']}")


messages = []
agent = my_agent

while True:
    user_input = input("> ")
    if user_input.strip().lower() == "bye" or user_input.strip().lower() == "q":
        break
    messages.append({"role": "user", "content": user_input})
    pretty_print_messages(messages)
    response = client.run(agent=agent, messages=messages)
    messages.append({"role": "assistant", "content": response.messages[-1]["content"]})
    agent = response.agent
    pretty_print_messages(messages)